In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import gspread_pandas
from gspread_pandas import Spread, Client

In [ ]:
#yf.download(tickers='CBA.AX', period='3d', #group_by=stocks)

In [ ]:
def download_data(stock, timeperiod):
    df = yf.download(tickers=stock, period=timeperiod)
    stockname = stock.replace('.AX','')
    df['Company'] = stockname
    return df

In [ ]:
stocks_list = ['ALL.AX', 'CBA.AX', 'WBC.AX', 'ANZ.AX', 'NAB.AX', 'APT.AX', 'Z1P.AX', 'WZR.AX', 'TLS.AX', 'MPL.AX', 'PBH.AX', 'BTH.AX', 'PPH.AX', 'ESK.AX', 'QFE.AX', 'AXE.AX', 'ID8.AX']

In [ ]:
df_raw = pd.concat([download_data(stocks, '120d') for stocks in stocks_list])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
#For checking purposes
# df_raw.query('Company == "WBC"').filter(['Open','High','Low','Close']).pct_change()

In [ ]:
#Create pivoted dataframe for close price
df_pivot_close = df_raw.reset_index().pivot(index='Date', columns='Company', values='Close')
df_pivot_close = df_pivot_close.reset_index().rename_axis(None, axis='columns')

In [ ]:
#Create pivoted dataframe for daily percentage change
df_pivot_pct = df_raw.reset_index().pivot(index='Date', columns='Company', values='Close').pct_change()
df_pivot_pct = df_pivot_pct.reset_index().rename_axis(None, axis='columns')

In [ ]:
#Create 'long' dataframe for close price
df_melt_close = df_pivot_close.melt(id_vars='Date', var_name='Company', value_name='ClosePrice')

In [ ]:
#Create 'long' dataframe for daily percentage change
df_melt_pct = df_pivot_pct.melt(id_vars='Date', var_name='Company', value_name='DailyPctDiff')

In [ ]:
#Join both daily percentage change and close price dataframes
df = df_melt_close.merge(df_melt_pct, how='inner', on=['Date','Company'])

In [ ]:
df.to_csv('asx_manual.csv', index=False)

In [ ]:
#Without access to google drive, below doesn't work
# spread = Spread('Shares', sheet='raw_data')
# spread.df_to_sheet(df, index=False, replace=True)